In [ ]:
import numpy as np
import pandas as pd

from bokeh.io import output_notebook, show
from bokeh.plotting import figure
from bokeh.models import tickers, ColumnDataSource
from bokeh.transform import linear_cmap
from bokeh.palettes import RdYlGn4

In [ ]:
output_notebook()

In [ ]:
debt = pd.read_excel("debt.xlsx", parse_dates=True)
debt

,Year,External Debt Stocks Total,GDP Growth,Debt Service on External Debt,Exports of Good and Service,Debt Service Ratio
0,2010,1.982784e+11,6.223854,3.156906e+10,1.834806e+11,0.172057
1,2011,2.196294e+11,6.169784,3.107341e+10,2.350951e+11,0.132174
2,2012,2.526229e+11,6.030051,3.723217e+10,2.257444e+11,0.164931
3,2013,2.636436e+11,5.557264,4.141707e+10,2.183084e+11,0.189718
4,2014,2.925652e+11,5.006668,6.011999e+10,2.108201e+11,0.285172
5,2015,3.077493e+11,4.876322,6.027260e+10,1.821583e+11,0.330880
6,2016,3.189422e+11,5.033069,6.440385e+10,1.778860e+11,0.362051
7,2017,3.535640e+11,5.069786,5.880692e+10,2.049245e+11,0.286969
8,2018,3.795890e+11,5.169706,5.546865e+10,2.185809e+11,0.253767
9,2019,4.020839e+11,5.024714,8.178026e+10,2.060152e+11,0.396962


In [ ]:
debt.dtypes

Year                               int64
External Debt Stocks Total       float64
GDP Growth                       float64
Debt Service on External Debt    float64
Exports of Good and Service      float64
Debt Service Ratio               float64
dtype: object

In [ ]:
# Data
def round_growth(value):
    """Round all growth into 2 digits decimal"""
    return round(value, 2)

def growth_x_offset(value):
    """Growth value lower than 5% will have text offset"""
    return 0 if value > 5. else -15.

def growth_y_offset(value):
    """
    Growth value lower than 5% will have text offset 25,
    other will have text offset -5,
    """
    return -5. if value > 5. else 25.

year = debt["Year"]
growth = debt["GDP Growth"].to_list()

g_round = [round_growth(g) for g in growth]
x_offset = [growth_x_offset(g) for g in growth]
y_offset = [growth_y_offset(g) for g in growth]

# Figure
p = figure(plot_height=400, plot_width=900, y_range=(4.5, 6.5),
           title="Pertumbuhan Ekonomi Indonesia Tahun 2010 - 2019 (%)")

# Plotting
p.vbar(x=year, top=growth, bottom=5., width=0.5, color="#FAAB66")
p.line(x=year, y=growth, line_width=2.5, color="#5989FA")
p.circle(x=year, y=growth, color="#FAEA4D", line_color="#5989FA", line_width=3., size=10.)
p.text(x=year, y=growth, text=g_round, x_offset=x_offset, y_offset=y_offset)

# Styling
#   xaxis
p.xaxis.axis_label = "Tahun"
p.xaxis.ticker = year

#   yaxis
p.yaxis.axis_label = "Persen"

#   title
p.title.align = "center"
p.title.text_font_size = "20px"


show(p)

In [ ]:
# Data
exdebt = debt["External Debt Stocks Total"].to_numpy() / 1e+9
exdebt_dense = np.linspace(0, max(exdebt), 50)
q_list = [0.25, 0.5, 0.75, 1.] # quantile
palette = RdYlGn4

# Figure
p2 = figure(plot_height=400, plot_width=900, y_range=(150, 450),
           title="Total Utang Luar Negeri Indonesia Tahun 2010 - 2019 (Miliar Dolar)")

# Color style
def color_mapper(value):
    """Map specific color for every value"""
    for x, y in zip(q_list, palette):
        quantile = np.quantile(exdebt, x)
        if value <= quantile:
            return y

# Plotting
for x, y in zip(year, exdebt):
    data = [value for value in exdebt_dense if value <= y]
    length = len(data)
    colors = [color_mapper(c) for c in data]
    p2.rect(x=[x] * length, y=data, width=.5, height=4., 
           fill_color=colors, line_color=None)

# Styling
#   xaxis
p2.xaxis.axis_label = "Tahun"
p2.xaxis.ticker = year

#   yaxis
p2.yaxis.axis_label = "Utang Luar Negeri (Miliar Dolar)"

#   grid
p2.xgrid.grid_line_color = None

#   title
p2.title.align = "center"
p2.title.text_font_size = "20px"

show(p2)